## LU Factorization (LU Decomposition)

Any non-singular matrix $A$ can be factored into a lower triangular matrix $L$, and uppoer triangular matrix $U$ using procedures we have already established with Gaussian elimination. This proves very useful for numerical computation and is, in fact, one of the most common ways most packaged linear algebra solvers solve non-sparse linear systems.

Previously we learned that by using Gaussian elimination we can solve the linear system $A\bar{x} = \bar{b}$ in $O(\frac{1}{3}n^3)$ arithmetic operations to determine $\bar{x}$. It turns out if $A$ has the form $A=LU$ we can solve for $\bar{x}$ using a two step process. First we let $\bar{y}=U\bar{x}$ and solve the system for $L\bar{y}=\bar{b}$ for $\bar{y}$. Since $L$ is lower triangular we use a forward substitiution process that only takes $O(n^2)$ operations. Once $\bar{y}$ is known, the upper triangular system $U\bar{x}=\bar{y}$ can be solved with back-substitution $O(n^2)$ operations. Therefore using this procedure we can reduce the solution of $A\bar{x} = \bar{b}$ from $O(\frac{1}{3}n^3)$ to $O(2n^2)$ operations. For large systems this can reduce the calculation time by more than 95%. Determining the $L$ and $U$ matrices still takes $O(\frac{1}{3}n^3)$, but it only has to be done for a single $A$ matrix and can be used efficiently to solve for $\bar{x}$ with many right-hand side vectors, $\bar{b}$.

We already know one way to transform $A$ into $U$ by using Gaussian elimination. We will leave the proof for linear algebra class, but we find L by performing negations of the same operations on the identity matrix in reverse order. For example when going $A\rightarrow U$ if we perform the row operation

$$
E_i - m_{ji}E_j\rightarrow E_i
$$

to undo this and return to $A$ we would perform

$$
E_i+m_{ji}E_j\rightarrow E_i
$$

by performing the second operation on the identity matrix with the same dimensions as $A$ we will end up with an $L$. I emphasize *an* here because this is only one way to decompose $A$ into $L$ and $U$, there are other methods and an infinite number of $L$ and $U$ matrices, (i.e. they are not unique).

#### LU Example

We will look at a simple example and write out the row operations.

$$
A = \begin{pmatrix}1&1&0\\2&1&-1\\3&-1&-1\end{pmatrix}
\begin{matrix}E_2-2E_1\\\rightarrow\end{matrix}
\begin{pmatrix}1&1&0\\0&-1&-1\\3&-1&-1\end{pmatrix}
\begin{matrix}E_3-3E_1\\\rightarrow\end{matrix}
\begin{pmatrix}1&1&0\\0&-1&-1\\0&-4&-1\end{pmatrix}
\begin{matrix}E_3-4E_2\\\rightarrow\end{matrix}
\begin{pmatrix}1&1&0\\0&-1&-1\\0&0&3\end{pmatrix} = U
$$

$$
I = \begin{pmatrix}1&0&0\\0&1&0\\0&0&1\end{pmatrix}
\begin{matrix}E_3+4E_2\\\rightarrow\end{matrix}
\begin{pmatrix}1&0&0\\0&1&0\\0&4&1\end{pmatrix}
\begin{matrix}E_3+3E_1\\\rightarrow\end{matrix}
\begin{pmatrix}1&0&0\\0&1&0\\3&4&1\end{pmatrix}
\begin{matrix}E_2+2E_1\\\rightarrow\end{matrix}
\begin{pmatrix}1&0&0\\2&1&0\\3&4&1\end{pmatrix} = L
$$

Let's check our results with *Python*.

In [3]:
L = [[1,0,0],[2,1,0],[3,4,1]]
U = [[1,1,0],[0,-1,-1],[0,0,3]]

TypeError: can't multiply sequence by non-int of type 'list'

In [10]:
from numpy import matrix
L = matrix([[1,0,0],[2,1,0],[3,4,1]])
U = matrix([[1,1,0],[0,-1,-1],[0,0,3]])
print L*U

[[ 1  1  0]
 [ 2  1 -1]
 [ 3 -1 -1]]


Notice that the entries in $L$ below the diagonal are simply the $m_{ij}$'s therefore we do not actually have to perform the row operations we can simply insert the $m_{ij}$ components into their proper place and move forward.

## Psuedocode for a simple LU factorization

Consider the matricies:

$$
A = \begin{pmatrix}a_{11}&a_{12}&...&a_{1n}\\a_{21}&\ddots& &a_{2n}\\\vdots& &\ddots& \vdots\\a_{n1}&...&...&a_{nn}\end{pmatrix},
L = \begin{pmatrix}l_{11}&l_{12}&...&l_{1n}\\l_{21}&\ddots& &l_{2n}\\\vdots& &\ddots& \vdots\\l_{n1}&...&...&l_{nn}\end{pmatrix},
U = \begin{pmatrix}u_{11}&u_{12}&...&u_{1n}\\u_{21}&\ddots& &u_{2n}\\\vdots& &\ddots& \vdots\\u_{n1}&...&...&u_{nn}\end{pmatrix}
$$

  1. Initialize $L$ to an identity matrix, $I$, of dimension $n$ by $n$ and $U = A$.
  1. For $i = 1$, ..., $n$ do Step A.
    1. For $j=i+1, ..., n$ do Steps a-b
      1. Set $I_{ji}=u_{ji}/u_{ii}$
      1. Perform $U_j = (U_j-I_{ji}U_i)$ (where $U_i, U_j$ represent the $i$ and $j$ rows of the matrix $U$, respectively)

In [25]:
import pprint
def mult_matrix(M,N):
    tuple_N = zip(*N) #Converts N into a list of tuples (ordered list of elements) of columns
    return [[sum(el_m*el_n for el_m, el_n in zip(row_m, col_n)) for col_n in tuple_N] for row_m in M]

def pivot_matrix(M):
    m = len(M)
    id_mat = [[float(i ==j) for i in xrange(m)] for j in xrange(m)] #Creates identity matrix for M
    for j in xrange(m):
        row = max(xrange(j,m), key=lambda i: abs(M[i][j])) #rearranges matrix so the largest elements of M are along the diagonal
        if j != row:
            id_mat[j], id_mat[row] = id_mat[row], id_mat[j] #swaps the rows
    return id_mat
def lu_decomposition(A):
    n = len(A)
    L = [[0.0]*n for i in xrange(n)] # create zero matrices for L 
    U = [[0.0]*n for i in xrange(n)] #and U
    
    P = pivot_matrix(A)   #Creates the pivot matrix P
    PA = mult_matrix(P,A) #and the multipled matrix PA
    
    for j in xrange(n): #Performs the LU Decomposition
        L[j][j] = 1.0 #all diagonal entries of L are set to unity
        for i in xrange(j+1):
            s1 = sum(U[k][j]*L[i][k] for k in xrange(i))
            U[i][j] = PA[i][j] - s1
        for i in xrange(j,n):
            s2 = sum(U[k][j]*L[i][k] for k in xrange(j))
            L[i][j] = (PA[i][j] - s2) / U[j][j]
    return (P, L, U)
A = [[1,1,0],[2,1,-1],[3,-1,-1]]
P, L, U = lu_decomposition(A)

print "A:"
pprint.pprint(A)
print "P:"
pprint.pprint(P)
print "L:"
pprint.pprint(L)
print "U:"
pprint.pprint(U)

A:
[[1, 1, 0], [2, 1, -1], [3, -1, -1]]
P:
[[0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [1.0, 0.0, 0.0]]
L:
[[1.0, 0.0, 0.0],
 [0.6666666666666666, 1.0, 0.0],
 [0.3333333333333333, 0.8, 1.0]]
U:
[[3.0, -1.0, -1.0],
 [0.0, 1.6666666666666665, -0.33333333333333337],
 [0.0, 0.0, 0.6000000000000001]]


In [1]:
import pprint
import numpy as np
import scipy.linalg as la #SciPy Linear Algebra Library with la as a shortcut

A = np.array([ [a11, a12, a13, a14],[a21, a22, a23, a24],[a31, a32, a33, a34],[a41, a42, a43, a44] ])
P, L, U = la.lu(A)

print "A:"
pprint.pprint(A)
print "P:"
pprint.pprint(P)
print "L:"
pprint.pprint(L)
print "U:"
pprint.pprint(U)

ImportError: No module named scipy.linalg

## Psuedocode for solving equations after LU factorization

Once we have $L$ and $U$ we can solve for as many right-hand side vectors $\bar{b}$ as desired very quickly using the following two step process. First we let $\bar{y}=U\bar{x}$ and then solve for $L\bar{y}=\bar{b}$ for $\bar{y}$ by using forward substitution. The pseudocode for this is as follows:

  1. Set $y_1 = b_1/I_{11}$
  1. For $i=2, 3, ..., n$ do Step 3.
  1. $y_i = (b_i - \sum\nolimits_{j=1}^{i-1} I_{ij}y_j)/I_{ii}$


Then solve $U\bar{x} = \bar{y}$ for $\bar{x}$ using backward substitution. The psuedocode for this is as follows:

  1. Set $x_n = y_n / u_{nn}$
  1. For $i = n-1, n-2, ..., 1$ do Step 3.
  1. $x_i = (y_i - \sum\nolimits_{j=i+1}^{n} u_{ij}x_j)/u_{ii}$

For a $PLU$ factorization would have the additional step of permutating the right-hand side vector $\bar{b} = P\bar{b}$ before doing the forward substitution to solve for $\bar{y}$.

## LU factorization, another look.

Let's condier a generic $LU$ factorization, for simplicity we will consider a set of 3 x 3 matrices, but these ideas will apply in the $n$x$n$ case.

$$
LU = \begin{pmatrix}I_{11}&0&0\\I_{21}&I_{22}&0\\I_{31}&I_{32}&I_{33}\end{pmatrix}
\begin{pmatrix}u_{11}&u_{12}&u_{13}\\0&u_{22}&u_{23}\\0&0&u_{33}\end{pmatrix} = 
\begin{pmatrix}a_{11}&a_{12}&a_{13}\\a_{21}&a_{22}&a_{23}\\a_{31}&a_{32}&a_{33}\end{pmatrix} = A
$$

If we multiply the two left side matrices together, we have

$$
\begin{pmatrix}I_{11}u_{11}&I_{11}u_{12}&I_{11}u_{12}\\I_{21}u_{11}&I_{21}u_{12}+I_{22}u_{22}&I_{21}u_{13}+I_{22}u_{23}\\I_{31}u_{11}&I_{31}u_{12}+I_{32}u_{22}&I_{31}u_{13}+I_{32}u_{23}+I_{33}u_{33}\end{pmatrix} = 
\begin{pmatrix}a_{11}&a_{12}&a_{13}\\a_{21}&a_{22}&a_{23}\\a_{31}&a_{32}&a_{33}\end{pmatrix}
$$

Now if we take the same approah as earlier where the $I_{ii}$'s are the 1's we can solve the first row of equations trivially, namely:

$u_{11} = a_{11}, u_{12} = a_{12}, u_{13} = a_{13},$ then we have enough information to solve the rest of the first column,

$I_{21} = a_{21}/a_{11}, I_{31}=a_{31}/a_{11},$ and the rest of the second row,

$u_{22} = (a_{22}-a_{21}^{2}/a_{11}), u_{23} = (a_{23}-a_{21}a_{23}/a_{11}),$ etc.

When this procedure is generalized it is known as the Doolittle alogrithm. There is a similar procedure known as Crout's method that uses the 1's on the diagonal of the $U$ matrix. The generalization of these methods will be shown in the upcoming pages describing their pseudocode.


## Pseudocode for Doolittle algorithm. 

Consider the matrices:

$$
A = \begin{pmatrix}a_{11}&a_{12}&...&a_{1n}\\a_{21}&\ddots& &a_{2n}\\\vdots& &\ddots& \vdots\\a_{n1}&...&...&a_{nn}\end{pmatrix},
L = \begin{pmatrix}l_{11}&l_{12}&...&l_{1n}\\l_{21}&\ddots& &l_{2n}\\\vdots& &\ddots& \vdots\\l_{n1}&...&...&l_{nn}\end{pmatrix},
U = \begin{pmatrix}u_{11}&u_{12}&...&u_{1n}\\u_{21}&\ddots& &u_{2n}\\\vdots& &\ddots& \vdots\\u_{n1}&...&...&u_{nn}\end{pmatrix}
$$

  1. For $k = 1, 2, ..., n$ do Steps 2-6
  1. Set $I_{kk}=1$
  1. For $j=k, k+1, ..., n$ do Step 4.
  1. $u_{kj} = a_{kj} - \sum\nolimits_{m=1}^{k-1}I_{km}u_{mj}$
  1. For $i=k+1, k+2, ..., n$ do Step 6.
  1. $I_{ik}=\frac{a_{ik}-\sum\nolimits_{m=1}^{k-1}I_{im}u_{mk}}{u_{kk}}$

## Pseudocode for Crout algorithm.

Consider the matrices:

$$
A = \begin{pmatrix}a_{11}&a_{12}&...&a_{1n}\\a_{21}&\ddots& &a_{2n}\\\vdots& &\ddots& \vdots\\a_{n1}&...&...&a_{nn}\end{pmatrix},
L = \begin{pmatrix}l_{11}&l_{12}&...&l_{1n}\\l_{21}&\ddots& &l_{2n}\\\vdots& &\ddots& \vdots\\l_{n1}&...&...&l_{nn}\end{pmatrix},
U = \begin{pmatrix}u_{11}&u_{12}&...&u_{1n}\\u_{21}&\ddots& &u_{2n}\\\vdots& &\ddots& \vdots\\u_{n1}&...&...&u_{nn}\end{pmatrix}
$$

  1. For $k = 1, 2, ..., n$ do Steps 2-6
  1. Set $I_{kk} = a_{kk}-\sum\nolimits_{m=1}^{k-1}I_{km}u_{mk}$
  1. For $j = k, k+1, ..., n$ do Step 4.
  1. $u_{kj} = \frac{a_{kj}-\sum\nolimits_{m=1}^{k-1}I_{km}u_{mj}}{I_{kk}}$
  1. For $i = k+1, k+2, ..., n$ do Step 6.
  1. $I_{ik} = \frac{a_{ik}-\sum\nolimits_{m=1}^{k-1}I_{im}u_{mk}}{u_{kk}}$

## Pseudocode for Cholesky decomposition.

If matrix $A$ is symmetric and positive definite, then there exists a lower triangular matrix $L$ such that $A=LL^T$. This is just a special case of the $LU$ decomposition, $U=L^T$. The algorithm is slightly simpler than the Doolittle or Crout methods.

Consider the matrices:

$$
A = \begin{pmatrix}a_{11}&a_{12}&...&a_{1n}\\a_{21}&\ddots& &a_{2n}\\\vdots& &\ddots& \vdots\\a_{n1}&...&...&a_{nn}\end{pmatrix},
L = \begin{pmatrix}l_{11}&l_{12}&...&l_{1n}\\l_{21}&\ddots& &l_{2n}\\\vdots& &\ddots& \vdots\\l_{n1}&...&...&l_{nn}\end{pmatrix},
$$

  1. For $k=1,2,...,n$ do Steps 2-4
  1. Set $I_{kk} = \sqrt{a_{kk}-\sum\nolimits_{m=1}^{k-1}I_{km}^2}$
  1. For $i=k+1,k+2,...,n$ do Step 4.
  1. $I_{ik}=\frac{a_{ik}-\sum\nolimits_{m=1}^{k-1}I_{im}I_{km}}{I_{kk}}$

##Solving for the inverse of $A$ once the $LU$ decomposition is known.

Once the $LU$ decomposition of $A$ is complete it is straightforward to find the inverse of $A$, using the same forward and backward substituion process we used when solving for an arbitrary right hand side vector $\bar{b}$. Except we will do the procedure $n$ times, where $n$ is the number of columns of $A$ and $\bar{b} = [\bar{b_1};\bar{b_2};...;\bar{b_n}] = I$ are the columns of the Identity matrix. Stated differently, we will use the columns of the identity matrix as individual right-hand side vectors, $\bar{b}$, in order to solve for the inverse column-by-column.

The pseudocode is as follows:

  1. Set $\bar{b} = I$ with dimension $n$x$n$
  1. For $k = 1, 2, ..., n$ do Steps 3-9
  1. Set $y_{1k}=\frac{b_{1k}}{I_{11}}$
  1. For $i = 2, 3, ..., n$ do Step 5.
  1. $y_{ik} = \frac{b_{ik}-\sum\nolimits_{j=1}^{i-1}I_{ij}y_{jk}}{I_{ii}}$
  1. Set $x_{nk} = \frac{y_{nk}}{u_{nn}}$
  1. For $i = n-1, n-2, ..., 1$ do Step 8.
  1. $x_{ik}= \frac{y_{ik}-\sum\nolimits_{j=i+1}^{n}u_{ij}x_{jk}}{u_{ii}}$

##Determinant of a Matrix.

You might reall from linear algebra that there are several ways of computing the determinant of a matrix (e.g. Leibniz formula, Laplace formula, Cramer's rule, etc.), however, none of these are computationally as efficient as using the $LU$ decomposition and a few properties of determinants to solve for the determinate of a matrix $A$. Recall,

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;For $A = LU, \rightarrow det(A) = det(L)det(U)$

also for an upper (or lower) triangular matrix. The determinate of the matrix is simply the product of the diagonal entries. Therefore, if we solve for $L$ and $U$ using the Doolittle method, where there are 1's on the diagonal of the $L$ matrix, then the determinate of $L$ is 1. Thus,

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$det(A) = 1*det(U) = \prod_{j=1}^{n}u_{jj}$

Similarly, for a $PLU$ decomposition,

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$det(A) = det(P)det(L)det(U) = det(P)*\prod_{j=1}^{n}u_{jj}$

but, $P$ is just a permutation of the Identity matrix. Let's observe what happens when we do five semi-random row permutations of the identity matrix and calculate the determinate of $P$ each time.

In [7]:
import numpy as np
from numpy import matrix
n = 3
P = np.matrix(np.identity(n))
print P


[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]


Therefore all we need to do is keep track of the number of row permutations and the $det(P)$ will have the following properties:

$det(P) = \{ _{ 1\,for\,even\,number\,of\,permutations}^{-1\,for\,odd\,number\,of\,permutations}$